# Chacha

In [1]:
from typing import Tuple, List
from scripts.utils import split_words, join_words, text_to_int, sum32

## Pad

In [2]:
_c = [0x61707865, 0x3320646e, 0x79622d32, 0x6b206574]

def pad(s: int, j: int, n: int) -> int:
    """Una función que combina una semilla s de 256 bits, un contador j de 64 bits (j0 y j1 de 32bits), y un ruido (nonce) n de 64 bits (n0 y n1 de 32 bits) para formar un bloque de 512 bits

    Args:
        s: Seed (256 bits, 8 palabras de 32 bits)
        j: counter (64 bits)
        n: Nonce (64 bits)

    Returns:
        512 bits (16 palabras de 32 bits)
    """

    s = split_words(s, 32 // 8, 8)
    j = split_words(j, 32 // 8, 2)
    n = split_words(n, 32 // 8, 2)

    return [
        _c[0], _c[1], _c[2], _c[3],
        s[0], s[1], s[2], s[3],
        s[4], s[5], s[6], s[7],
        j[0], j[1], n[0], n[1]
    ]

In [3]:
assert(
    pad(0x47f515b1dd45f8d5aceea73b52971be21f7b4b3355a35fd6a2799898ed2f8c97,
    0x722d9d570ac23201,
    0xed539cd99e1d2f20) ==
    
    [0x61707865, 0x3320646e, 0x79622d32, 0x6b206574,
    0x47f515b1, 0xdd45f8d5, 0xaceea73b, 0x52971be2,
    0x1f7b4b33, 0x55a35fd6, 0xa2799898, 0xed2f8c97,
    0x722d9d57, 0x0ac23201, 0xed539cd9, 0x9e1d2f20]
), "Error"

## Permutación publica

### Cuarto de rondas

In [4]:
def rot(w: int, r: int) -> int: 
    """No se queda hace

    Args:
        w: palabra 
        r: No me acuerdo
    """

    mask = 0xffffffff
    return ((w << r) & mask) | (w >> (32 - r))

def QR(a: int, b: int, c: int, d: int) -> Tuple[int, int, int, int]:
    """Cuarto de ronda

    Args:
        a (int): palabra
        b (int): palabra
        c (int): palabra
        d (int): palabra

    Returns:
        Tuple[int, int, int, int]: sequencia de 4 palabras
    """
    a = sum32(a, b)
    d = d ^ a 
    d = rot(d, 16)

    c = sum32(c, d)
    b = b ^ c
    b = rot(b, 12)

    a = sum32(a, b)
    d = d ^ a
    d = rot(d, 8)

    c = sum32(c, d)
    b = b ^ c
    b = rot(b, 7)
    
    return a, b, c, d

In [5]:
assert(
    QR(0xc2619378, 0xecdaec96, 0xe62bd0c8, 0x2b61be56) ==
    (0x9ad7bc93, 0x130fa62c, 0xb3bd23c9, 0x8f38cea4)
), "Error"

### Ronda

In [6]:
def round(i):
    i[0], i[4], i[8], i[12] = QR(i[0], i[4], i[8], i[12])
    i[1], i[5], i[9], i[13] = QR(i[1], i[5], i[9], i[13])
    i[2], i[6], i[10], i[14] = QR(i[2], i[6], i[10], i[14])
    i[3], i[7], i[11], i[15] = QR(i[3], i[7], i[11], i[15])

    i[0], i[5], i[10], i[15] = QR(i[0], i[5], i[10], i[15])
    i[1], i[6], i[11], i[12] = QR(i[1], i[6], i[11], i[12])
    i[2], i[7], i[8], i[13] = QR(i[2], i[7], i[8], i[13])
    i[3], i[4], i[9], i[14] = QR(i[3], i[4], i[9], i[14])

    return i

### Permutación

In [7]:

def perm(x: int, ROUNDS: int = 20): 
    """Permutación

    Args:
        x: 512 bits (16 palabras 32 bits)
        ROUNDS: Número de rondas
            - 20 (ChaCha20/20)
            - 8 (ChaCha20/8)
            - 12 (ChaCha20/12)
            
    Returns:
        512 bits (16 palabras 32 bits)
    """
    assert(ROUNDS in [8, 12, 20]), "Invalid number of rounds"
    
    i = x.copy()
    
    for _ in range(0, ROUNDS, 2): 
        round(i)
    return i


In [8]:
s = 0x47f515b1dd45f8d5aceea73b52971be21f7b4b3355a35fd6a2799898ed2f8c97
j = 0x722d9d570ac23201
n = 0xed539cd99e1d2f20

_pad = pad(s, j, n)

assert(join_words(perm(_pad), 32 // 8) == 0xc9ead123f6eee2042ce8442128342dcdddec68c9446ec082de92f642f498c0a843d9d8d27c44c2bd1945edeb3411fa78fffb0e607ec9ec17b7a5cfae23663818)

## Generador psudoaleatorio

In [9]:
L = 1 # 
def G(s: int, n: int):
    r = [0]*L

    for j in range(L):
        h = pad(s, j, n) # 512 bits
        pi = perm(h) # 512 bits

        r[j] = [0]*16
        for i in range(16):
           r[j] = join_words(pi, 32) + join_words(h, 32)
           
    return r

## Cifrado y decifrado

In [10]:
k = G(0x112233445566778899aabbccddeeff00, 0x0123456789abcdef)[0]
m = text_to_int("Hola mundo")
c = m ^ k
assert(c ^ k == m), "Error"

## References
- Class notes 
- [ChaCha, a variant of Salsa20 - Daniel J. Bernstein](https://cr.yp.to/chacha/chacha-20080128.pdf)